<a href="https://colab.research.google.com/github/eakubrakova/MFTI_2023/blob/main/melb_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import pandas as pd
melb_data_fe = pd.read_csv('/content/melb_data_fe.csv', sep = ',')
melb_df = melb_data_fe.copy()
# удалим дублирующиеся признаки

#посчитаем общее количество комнат
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
display(total_rooms)

#получим средний метраж комнаты, как отношение общей площади здания к количеству комнат
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'])

# вычтем из площади застройки площадь земельного участка
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']

#вычислим соотношение площади здания к общей площади земельного участка
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])

# стандартизируем отображение даты при помощи datetime и выделим квартал (quarter)
# продажи объектов недвижимости

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
quarters = melb_df['Date'].dt.quarter

# Найдем второй по популярности квартал продажи.
# В качестве ответа запишите число объектов, проданных в этом квартале.
print(quarters.value_counts().iloc[1])


# узнаем в какой день недели было больше всего продаж
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
print(weekend_count)

# Функция get_weekend(weekday) принимает на вход элемент столбца WeekdaySale и
# возвращает 1, если день является выходным, и 0 — в противном случае, создадим столбец Weekend в таблице melb_df с помощью неё.
# Применим эту функцию к столбцу и вычислите среднюю цену объекта недвижимости,
# проданного в выходные дни.
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
print(round(melb_df[melb_df['Weekend']==1]['Price'].mean(), 2))

"""Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df
следующим образом: оставьте в столбце только 49 самых популярных компаний,
 а остальные обозначьте как 'other'.
Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson',
больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'.
Ответ округлите до десятых."""

popular_seller = melb_df['SellerG'].value_counts().nlargest(49).index
# заменяем имена компаний, не попавших в список популярных, на строку 'other'
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other')

a = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
b = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
print(round(a/b, 1))

# преобразуем ряд признаков в категориальные
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

print(melb_df['Regionname'].cat.categories)
# переименуем категории признака типа постройки Type
# — заменим их на полные названия (напомним, u — unit, h — house, t — townhouse).

melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

# добавим новую категорию flat
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
# заменяем имена пригородов, не попавших в список популярных, на строку 'other'
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category') # преобразуем тип столбца
melb_df.info()

#Отсортируем таблицу по цене объектов и выведем первые 10 строк

melb_df.sort_values(by='Price').head(10)

# отсортируем таблицу по убыванию (от самой последней до самой первой) даты продажи объекта (Date)\
melb_df.sort_values(by='Date', ascending=False)

melb_df.sort_values(by=['Distance', 'Price']).loc[:, ['Distance', 'Price']]

# Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath,
#у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8.
# Результат отсортируем по дате продажи (Date) в порядке возрастания,
# а после проведём сортировку по убыванию коэффициента соотношения площадей.
#Также обновим старые индексы на новые, установив параметр ignore_index на True.
#Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio

"""mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]"""

#Произведите сортировку столбца AreaRatio по убыванию.
#При этом индексы полученной таблицы замените на новые.
# Какое значение площади здания находится в строке 1558?
#Ответ округлите до целого числа
melb_df.sort_values(
    by = ['AreaRatio'],
    ascending=False,
    ignore_index = False
    ).loc[[1558],['BuildingArea']]

# Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2.
#Отсортируйте полученную таблицу сначала по возрастанию числа комнат,
#а затем по убыванию средней площади комнат (MeanRoomsSquare). Индексы таблицы замените на новые.
# Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
int(melb_df[mask1&mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price'])

# Cгруппируем данные по столбцу Type и применим агрегирующую функцию среднего:
melb_df.groupby(by='Type').mean(numeric_only=True)
# агрегирующие методы можно применять только к интересующему нас столбцу.
# Например,сравним средние цены на объекты в зависимости от их типа:

melb_df.groupby('Type')['Price'].mean()

# выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.
#Для этого найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона.
#Результат отсортируем по убыванию расстояния:

melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

# построим таблицу для анализа продаж по месяцам.
#Для этого найдём количество продаж, а также среднее и максимальное значения
# цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale).
# Результат отсортируем по количеству продаж в порядке убывания:

melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

# После базовых математических функций наиболее частым агрегированием является подсчёт
#числа уникальных значений. Так, например, мы можем вычислить число уникальных риелторских компаний
#в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше.

melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

#Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе.
#В качестве ответа запишите количество комнат, для которых средняя цена наибольшая

grouped = melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)
grouped

# Какой регион имеет наименьшее стандартное отклонение по географической широте (Lattitude)?

melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

#Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября
#(включительно) 2017 года?
#Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.

date1 = '2017-05-01'
date2 = '2017-09-01'
mask = (date1 <= melb_df['Date']) & (melb_df['Date']<= date2)
melb_df[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)

melb_df.groupby('SellerG')['Price'].sum().sort_values(ascending=True)

#построим ту же самую таблицу, но уже с использованием метода pivot_table.
#В качестве параметра values укажем столбец Price, в качестве индексов сводной таблицы возьмём Rooms,
# а в качестве столбцов — Type. Агрегирующую функцию оставим по умолчанию (среднее).
# Дополнительно заменим пропуски в таблице на значение 0.
# Финальный результат для наглядности вывода округлим с помощью метода round() до целых.

melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

#проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной.
#Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname),
# а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1,
# если день был выходным, и 0 — в противном случае.
# В качестве значений сводной таблицы возьмём количество продаж.

melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

#найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type)
# и его региона (Regionname). Чтобы посмотреть несколько статистических параметров,
#нужно передать в аргумент aggfunc список из агрегирующих функций.
#Построим такую сводную таблицу, где пропущенные значения заменим на 0:

melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

#Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания
# от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms).
# Для какой комбинации признаков площадь здания наибольшая?

pivot = melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median',
    fill_value=0
)
display(pivot)

#Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (Price)
# от риелторского агентства (SellerG) и типа здания (Type).
#Во вновь созданной таблице найдите агентство, у которого медианная цена
#для зданий типа unit максимальна.

pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
)
max_unit_price = pivot['unit'].max()
print(pivot[pivot['unit'] == max_unit_price].index[0])

0         5
1         5
2         8
3         8
4         8
         ..
13575    10
13576     8
13577     8
13578     9
13579     9
Length: 13580, dtype: int64

0        25.200000
1        15.800000
2        18.750000
3        15.750000
4        17.750000
           ...    
13575    12.600000
13576    16.625000
13577    15.750000
13578    17.444444
13579    12.444444
Name: MeanRoomsSquare, Length: 13580, dtype: float64

0       -0.231707
1       -0.327660
2        0.056338
3        0.145455
4        0.083969
           ...   
13575   -0.676093
13576   -0.429185
13577   -0.551601
13578   -0.693060
13579   -0.527426
Name: AreaRatio, Length: 13580, dtype: float64

4359
9226
1083291.7
1.3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea   

None

...

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126,126,141.0,177,126,216.5,126,126
townhouse,88.0,114,126,159.5,152,0,0.0,0,0
unit,69.5,110,126,126.0,0,171,0.0,126,0


Nick
